### In this file we load the labels from JSON ground truth labels for each image and generate QA pairs, 2 pairs per image.

In [ ]:
import os
from dotenv import load_dotenv
from typing import List, Optional
from openai import OpenAI
from datasets import load_dataset
from datetime import date, time, datetime, timedelta
from dataclasses import dataclass

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json
from transformers import AutoModelForCausalLM

In [ ]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/24/cf/1fbe3eaf1d90796722c3dcd268e103a6a0918f25ec51138c7d27aec7f001/openai-1.55.1-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/65/ae/d503db7183fdc46295e8e88a4cbb71438e7f959c5635b03c5e839b4f3d33/jiter-0.7.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.11

#### Load dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("naver-clova-ix/cord-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)-00000-of-00004-4ecdadef88e06e74.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

(…)-00001-of-00004-746b6acd0207b060.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

...

(…)-00000-of-00001-9031a89e666533f9.parquet:   0%|          | 0.00/242M [00:00<?, ?B/s]

(…)-00000-of-00001-f6c27485d73a0328.parquet:   0%|          | 0.00/234M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
load_dotenv('vars.env', override=True)
BASE_URL = os.getenv("CS4973_BASE_URL")
API_KEY = os.getenv("CS4973_API_KEY")

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

In [ ]:
from huggingface_hub import login

login(token='')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Check if CUDA is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Move model to GPU
# model.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

#### Extract info avaliable about receipt in the dataset

In [ ]:
import json

def extract_relevant_info(ground_truth):
    if isinstance(ground_truth, str):
        ground_truth = json.loads(ground_truth)

    gt_parse = ground_truth.get("gt_parse", {})
    menu_items = gt_parse.get("menu", [])
    subtotal = gt_parse.get("sub_total", {})
    total = gt_parse.get("total", {})

    items_info = []
    for item in menu_items:
        if isinstance(item, dict):
            name = item.get("nm", "Unknown Item")
            quantity = item.get("cnt", "Unknown Quantity")
            price = item.get("price", "Unknown Price")
            items_info.append(f"Item: {name}, Quantity: {quantity}, Price: {price}")

    subtotal_price = "Unknown"
    tax_price = "Unknown"
    service_price = "Unknown"
    total_price = "Unknown"

    if isinstance(subtotal, dict):
        subtotal_price = subtotal.get("subtotal_price", "Unknown")
        tax_price = subtotal.get("tax_price", "Unknown")
        service_price = subtotal.get("service_price", "Unknown")

    if isinstance(total, dict):
        total_price = total.get("total_price", "Unknown")

    info = "\n".join(items_info)
    info += f"\nSubtotal: {subtotal_price}, Tax: {tax_price}, Service Charge: {service_price}, Total: {total_price}"
    return info


#### Generate Q&A pair

In [ ]:
def generate_single_qa_pair(receipt_info, client, previous_questions, max_content_length=1500):
    trimmed_content = receipt_info[:max_content_length]
    previous_questions_text = "\n".join(f"- {q}" for q in previous_questions)
    prompt = (
        f"You are an assistant specializing in analyzing receipts and generating questions and answers. "
        f"Here is information extracted from a receipt:\n{trimmed_content}\n\n"
        "### Guidelines for QA Generation\n"
        "- Create a question that is relevant to the provided receipt information.\n"
        "- Questions can reflect common customer concerns, such as pricing, quantity discrepancies, tax calculations, or general inquiries.\n"
        "- Generate answers that are **only one number** derived directly from or inferred based on the receipt information.\n"
        "- Ensure the question and answer are concise and relevant.\n"
        "- Avoid duplication of previous questions.\n\n"
        "### Previous Questions (do not duplicate these):\n"
        f"{previous_questions_text}\n\n"
        "### Generate exactly one question-answer pair in the following format:\n"
        "**Q:** [Generated question]\n"
        "**A:** [Single numeric answer]\n"
    )

    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt
        }],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        temperature=0.7
    )

    return response.choices[0].message.content.strip()



In [ ]:
# dataset = load_dataset("naver-clova-ix/cord-v1")
dataset = ds

i = 0
qa_pairs = []
for example in dataset['train']:
    ground_truth = example['ground_truth']
    image_name = example['image']
    receipt_info = extract_relevant_info(ground_truth)

    if not receipt_info.strip():
        continue

    previous_questions = []

    for _ in range(2):
        qa_text = generate_single_qa_pair(receipt_info, client, previous_questions)

        if 'A:' in qa_text:
            question, answer = qa_text.split('A:', 1)
            question = question.strip()
            answer = answer.strip()

            qa_pairs.append({
                "question": question,
                "answer": answer,
                "receipt_info": receipt_info,
                "image_name": image_name
            })
            print(f"QA Pair {i + 1}:")
            print("Question:", question)
            print("Answer:", answer)
            i += 1

            previous_questions.append(question)
            if len(previous_questions) > 3:
                previous_questions.pop(0)

Выходные данные были обрезаны до нескольких последних строк (5000).
Answer: ** 0 (Since the total bill is 18,000 and the sum of the item prices is 43,000, the service charge is 0 in this case as the subtotal is unknown.)
QA Pair 353:
Question: **Q:** What is the price of the Spaghetti Bini Muda (Bol ognese) dish?
**
Answer: ** 19.000
QA Pair 354:
Question: **Q:** What is the total price of the French Fries?
**
Answer: ** 12.000
QA Pair 355:
Question: **Q:** What is the total tax amount added to the subtotal?
**
Answer: ** 12,500
QA Pair 356:
Question: **Q:** What is the price per item of MIE KOCOK?
**
Answer: ** 29,000
QA Pair 357:
Question: **Q:** What was the price of the GARISH PC item on the receipt?
**
Answer: ** 259.000
QA Pair 358:
Question: **Q:** What was the total price before any tax or service charges were added?
**
Answer: ** 259.000
QA Pair 359:
Question: **Q:** What is the total service charge applied to the transaction?
**
Answer: ** 7,658
QA Pair 360:
Question: **Q:** 

In [ ]:
import json
import re

def save_cleaned_qa_pairs(qa_pairs, output_path='qa_dataset.json'):
    cleaned_qa_pairs = []

    for qa in qa_pairs:
        question = qa["question"]
        answer = qa["answer"]

        if answer.startswith("Answer: **"):
            answer = answer.replace("Answer: **", "").strip()

        answer = re.sub(r"[^\d,\.]", "", answer)
        cleaned_qa_pairs.append({"question": question, "answer": answer})

    with open(output_path, 'w') as output_file:
        json.dump(cleaned_qa_pairs, output_file, indent=4)

    print(f"Cleaned QA pairs saved to {output_path}")


In [ ]:
save_cleaned_qa_pairs(qa_pairs, output_path='qa_dataset.json')


Cleaned QA pairs saved to qa_dataset.json


In [ ]:
qa_pairs

[{'question': "**Q:**  What is the total cost of the 'Tahu Goreng' item after considering its quantity and price on the receipt?\n**",
  'answer': "**  The total cost of the 'Tahu Goreng' item after considering its quantity and price on the receipt is 36,000 * 2 = 72,000.",
  'receipt_info': 'Item: Nasi Campur Bali, Quantity: x 1, Price: 75,000\nItem: Bbk Bengil Nasi, Quantity: x 1, Price: 125,000\nItem: MilkShake Starwb, Quantity: x 1, Price: 37,000\nItem: Ice Lemon Tea, Quantity: x 1, Price: 24,000\nItem: Nasi Ayam Dewata, Quantity: x 1, Price: 70,000\nItem: Free Ice Tea, Quantity: x 3, Price: 0\nItem: Organic Green Sa, Quantity: x 1, Price: 65,000\nItem: Ice Tea, Quantity: x 1, Price: 18,000\nItem: Ice Orange, Quantity: x 1, Price: 29,000\nItem: Ayam Suir Bali, Quantity: x 1, Price: 85,000\nItem: Tahu Goreng, Quantity: x 2, Price: 36,000\nItem: Tempe Goreng, Quantity: x 2, Price: 36,000\nItem: Tahu Telor Asin, Quantity: x 1, Price: 40,000.\nItem: Nasi Goreng Samb, Quantity: x 1, Pri

In [ ]:
from PIL.PngImagePlugin import PngImageFile

# Fix the `image_name` field in `qa_pairs`
for qa in qa_pairs:
    image_name = qa.get('image_name')
    if isinstance(image_name, PngImageFile):  # Check if it's a PIL image object
        qa['image_name'] = image_name.filename  # Use the file name from the PIL object
    elif not isinstance(image_name, str):  # If it's not a string, convert it to a string
        qa['image_name'] = str(image_name)

# Save the fixed `qa_pairs` to a JSON file
output_path = 'qa_dataset_fixed.json'
with open(output_path, 'w') as output_file:
    json.dump(qa_pairs, output_file, indent=4)

print(f"Fixed QA dataset saved at: {output_path}")


Fixed QA dataset saved at: qa_dataset_fixed.json
